In [1]:
import urllib.request
import time
import sys
import getopt
import pandas as pd
import numpy as np
import pickle


In [21]:
%run utils.ipynb

In [38]:
embSize = 300
d1_emb_size=10
d2_emb_size=10
trainFile='../data/GAD_merged_samples_mesh.csv'

# Replace with path of word embdding file   
wefile = "/mnt/admin/GDA_backup/Dataset/embeddings/crawl-300d-2M.vec"
random_seed=1331

# Read Data

In [23]:
sent_contents, entity1_list, entity2_list, sent_lables,gene_id_list,disease_id_list,gene_symbol_list = dataRead_befree(trainFile)

Input File Reading


In [42]:
word_list, distance1_list, distance2_list  = get_wordList_and_distances_befree(sent_contents)

In [25]:
print ("train_size", len(word_list))


train_size 5330


In [26]:
sent_lengths= findSentLengths([word_list])

In [27]:
sentMax =max(max(sent_lengths[:]))


## Prepare Lable Matrix

In [28]:
# Three Class

# Y : positive association 
# N:  negative association
# F:  No Smmantic Association


label_dict = {'F':0, 'Y': 1, 'N': 2}


Y = mapLabelToId_befree(sent_lables, label_dict)
Y_train = np.zeros((len(Y), len(label_dict)))
for i in range(len(Y)):
    Y_train[i][Y[i]] = 1.0

# Generate Word and Position Embedding Vectors

### Word Embedding

In [39]:
word_dict, word_to_id, id_to_word = word_mapping(word_list)
print( "word dictonary length", len(word_dict))
word_vectors = readWordEmb_fastText(word_dict,id_to_word,word_to_id, wefile, embSize)
X_train =   mapWordToId(word_list, word_to_id)

Found 6766 unique words (139634 in total)
word dictonary length 6766
Reading word vectors
number of unknown word in word embedding 1525
number of known word in word embedding 5241


### Position Embedding

In [43]:
distance1_dict = makeDistanceList([distance1_list])
distance2_dict = makeDistanceList([distance2_list])
distance1_vectors = mapWordToId_list(distance1_list, distance1_dict)
distance2_vectors = mapWordToId_list(distance2_list, distance2_dict)

### Pad Embdding Vectors

In [44]:
X_train, distance1_vectors, distance2_vectors = paddData([X_train, distance1_vectors, distance2_vectors ], sentMax,padd_num= 100000) 


# Save Prepared Data as Pickle File

In [45]:
with open('data/pickles/befree_3class_crawl-300d-2M.pickle', 'wb') as handle:
    pickle.dump(Tr_gene_id_list, handle)
    pickle.dump(Tr_gene_symbol_list, handle)
    pickle.dump(Tr_disease_id_list, handle)
    pickle.dump(X_train, handle)
    pickle.dump(distance1_vectors, handle)
    pickle.dump(distance2_vectors, handle)    
    pickle.dump(Y_train, handle)
    pickle.dump(word_list, handle)
    pickle.dump(word_vectors, handle)
    pickle.dump(word_dict, handle)
    pickle.dump(distance1_dict, handle)
    pickle.dump(distance2_dict, handle)
    pickle.dump(label_dict, handle) 
    pickle.dump(sentMax, handle)
    
    
    
    